In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd# data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import socket, warnings
try:
    socket.setdefaulttimeout(1)
    socket.socket(socket.AF_INET, socket.SOCK_STREAM).connect(("1.1.1.1", 53))
except socket.error as ex: raise Exception("STOP")

In [ ]:
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

if iskaggle:
    !pip install -Uqq fastai duckduckgo_search

In [ ]:
! pip install fastbook
! [ -e /content ] && pip install -Uqq fastbook

In [ ]:
import fastbook
fastbook.setup_book()

In [ ]:
#hide
from fastbook import *
from fastai.vision.widgets import *

In [ ]:
search_images_ddg

In [ ]:
searches = 'moose', 'deer', 'elk'
path = Path('trail_cam')
from time import sleep

In [ ]:
for o in searches:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls = search_images_ddg(f'{o} photo'))
    sleep(10)
    resize_images(path/o, max_size=400, dest=path/o)

In [ ]:
fns = get_image_files(path)
fns

In [ ]:
failed = verify_images(fns)
failed.map(Path.unlink)
len(failed)

In [ ]:
from PIL import Image

deers = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct = 0.2 , seed = 42),
    get_y=parent_label,
    item_tfms=RandomResizedCrop(224, min_scale=0.5)
)

dls = deers.dataloaders(path)

In [ ]:
dls.valid.show_batch(max_n=6, nrows=2)

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(6, nrows=1)

In [ ]:
cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
for idx in cleaner.delete():
    #cleaner.fns[idx].unlink()
    
for idx,cat in cleaner.change(): 
    #shutil.move(str(cleaner.fns[idx]), path/cat)

In [ ]:
learn = vision_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(5)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
learn.export('model1.pkl')

In [ ]:
from fastdownload import download_url
urls = search_images_ddg('deer')
urls[1]
download_url(urls[1], 'deer.jpg')

im = Image.open('deer.jpg')
im.to_thumb(190,190)

In [ ]:
urls = search_images_ddg('elk')
urls[0]
download_url(urls[0], 'elk.jpg')

im = Image.open('elk.jpg')
im.to_thumb(190,190)

In [ ]:
urls = search_images_ddg('moose')
urls[1]
download_url(urls[1], 'moose.jpg')

im = Image.open('moose.jpg')
im.to_thumb(190,190)

In [ ]:
! pip install gradio

In [ ]:
im = PILImage.create("elk.jpg")
im.thumbnail((192, 192))
im

In [ ]:
from fastai.vision.all import *
import gradio as gr

learn = load_learner('model.pkl')
categories = ('deer', 'elk', 'moose')

def classify_image(img):
    pred, idx, probs = learn.predict(img)
    return dict(zip(categories, map(float,probs)))


In [ ]:
classify_image(im)

In [ ]:
image = gr.inputs.Image(shape=(192,192))
label = gr.outputs.Label()
examples = ['deer.jpg', 'elk.jpg', 'moose.jpg']

intf = gr.Interface(fn=classify_image, inputs=image, outputs=label, examples=examples)
intf.launch(inline=False, share=True)